In [1]:
!python make_dataset.py
!python train_model.py

[{'len': 990, 'name': '<cascade.data.pickler.Pickler', 'type': 'dataset'},
 {'len': 990,
  'n_features': 2,
  'name': '<cascade.data.range_sampler.RangeSampler',
  'type': 'dataset'},
 {'name': '<utils.utils.MakeClassificationDataset', 'type': 'dataset'}]
[{'len': 10, 'name': '<cascade.data.pickler.Pickler', 'type': 'dataset'},
 {'len': 10,
  'name': '<cascade.data.range_sampler.RangeSampler',
  'type': 'dataset'},
 {'name': '<utils.utils.MakeClassificationDataset', 'type': 'dataset'}]


The system cannot find the path specified.


[{'name': '<cascade.utils.sk_model.SkModel object at 0x00000240D6DCEB80>', 'created_at': DateTime(2022, 12, 4, 10, 55, 0, 720179, tzinfo=Timezone('UTC')), 'metrics': {'f1': 0.9644268774703558}, 'params': {}, 'type': 'model', 'pipeline': "Pipeline(steps=[('0', SVC(probability=True))])"}]


The system cannot find the path specified.


In [1]:
from cascade.models import ModelRepo
from cascade.meta import MetricViewer
from cascade import data as cdd

In [2]:
from correctness import correctness
from continuity import continuity
from contrastivity import contrastivity
from coherence import coherence

c:\xaib-experiments\venv3.8\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from xaib.explainers.feature_importance.constant_explainer import ConstantExplainer
from xaib.explainers.feature_importance.random_explainer import RandomExplainer
from xaib.explainers.feature_importance.shap_explainer import ShapExplainer
from xaib.explainers.feature_importance.lime_explainer import LimeExplainer

In [4]:
BS = 20

In [5]:
# Overwrite previous run
ModelRepo('repo', overwrite=True)

ModelRepo in repo of 0 lines

In [6]:
train_ds = cdd.Pickler('train_ds')

In [7]:
explainers = {
    'const': ConstantExplainer(n_features=2, constant=1),
    'random': RandomExplainer(n_features=2, shift=-15, magnitude=10),
    'shap': ShapExplainer(train_ds),
    'lime': LimeExplainer(train_ds, labels=(0, 1))
}

In [8]:
correctness(explainers, batch_size=BS)

100%|██████████| 1/1 [00:02<00:00,  2.18s/it]


In [9]:
continuity(explainers, batch_size=BS)

1it [00:00, 499.80it/s]
1it [00:00, 500.99it/s]
1it [00:00,  1.91it/s]
1it [00:03,  3.27s/it]


In [10]:
contrastivity(explainers, batch_size=BS)

100%|██████████| 1/1 [00:00<00:00,  1.13it/s]


In [11]:
coherence(explainers, batch_size=BS)

100%|██████████| 1/1 [00:02<00:00,  2.06s/it]


In [12]:
repo = ModelRepo('repo')

In [13]:
t = MetricViewer(repo).table

In [14]:
def present_metrics(case):
    return t[t['line'].apply(lambda x: case in x)].dropna(axis=1)

In [15]:
present_metrics('correctness')

,line,num,created_at,saved,name,parameter_randomization_check
12,repo\correctness,0,2022-12-04 10:58:43.760043+00:00,a few seconds after,const,0.000000
13,repo\correctness,1,2022-12-04 10:58:43.760043+00:00,a few seconds after,random,0.660232
14,repo\correctness,2,2022-12-04 10:58:43.760043+00:00,a few seconds after,shap,0.646809
15,repo\correctness,3,2022-12-04 10:58:43.760043+00:00,11 seconds after,lime,0.725573


In [16]:
present_metrics('continuity')

,line,num,created_at,saved,name,small_noise_check,multiplier
4,repo\continuity,0,2022-12-04 10:58:54.916436+00:00,a few seconds after,const,0.000000,0.01
5,repo\continuity,1,2022-12-04 10:58:54.916436+00:00,a few seconds after,random,0.558659,0.01
6,repo\continuity,2,2022-12-04 10:58:54.916436+00:00,a few seconds after,shap,0.000000,0.01
7,repo\continuity,3,2022-12-04 10:58:54.916436+00:00,a few seconds after,lime,0.030262,0.01


In [17]:
present_metrics('contrastivity')

,line,num,created_at,saved,name,label_difference
8,repo\contrastivity,0,2022-12-04 10:58:58.824174+00:00,a few seconds after,const,0.000000
9,repo\contrastivity,1,2022-12-04 10:58:58.824174+00:00,a few seconds after,random,0.616518
10,repo\contrastivity,2,2022-12-04 10:58:58.824174+00:00,a few seconds after,shap,1.414214
11,repo\contrastivity,3,2022-12-04 10:58:58.824174+00:00,a few seconds after,lime,0.064065


In [18]:
present_metrics('coherence')

,line,num,created_at,saved,other_disagreement,name
0,repo\coherence,0,2022-12-04 10:59:00.792574+00:00,a few seconds after,0.892064,const
1,repo\coherence,1,2022-12-04 10:59:00.792574+00:00,a few seconds after,0.707079,random
2,repo\coherence,2,2022-12-04 10:59:00.792574+00:00,a few seconds after,0.767700,shap
3,repo\coherence,3,2022-12-04 10:59:00.792574+00:00,a few seconds after,0.863103,lime
